# Evaluation using Precision, Recall, and F-score

In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize

## Load Target File

In [2]:
# file list in folder
folder_root = "data set"
sub_folder = "acuan hasil akhir"
directory_path = os.path.join(folder_root,sub_folder) # join folder root and label
target_files = sorted(os.listdir(directory_path)) # get file list with sorted

list_documents = {}
list_query = []

# iterate over target files
for target_file in target_files:

   # check .txt only
   if target_file.lower().endswith('.txt'):

      # read text
      file_name = os.path.join(directory_path,target_file)
      file = open(file_name, "r")
      filedata = file.read()

      # get query from file name
      index_of_dot = target_file.index('.')
      query = target_file[:index_of_dot]
      list_query.append(query)
      # add dictionary with key is query and the value is the text
      list_documents[query] = filedata 

In [3]:
list_documents['dabiru']

'Kata dabiru, yang diterjemahkan dengan ke ujung-ujungnya, pada mulanya berarti belakang.  Ayat ini menggunakan kata dabiru untuk menjelaskan bahwa semua dibinasakan Allah swt. termasuk siapa yang ada di belakang karena, buat Allah, belakang dan depan sama saja dari segi kemampuan-Nya membinasakan siapa yang dikehendaki-Nya. Kata qatha\'na dabiru /membinasakan sampai akhir dari segi harfiah berarti "Kami memutus belakang". Dalam peperangan, pasukan merupakan satu kesatuan, tetapi dengan kekalahan, mereka kocar-kacir dan lari pontang-panting. Siksa dimaksud adalah siksa pemunahan sedang keberadaan beliau bukanhanya dalam arti fisik tetapi dapat juga di pahami dalam arti keberadaan ajarannya. Kata li yaqtha\'a dabiru/ membinasakan sampai akhir dari segi harfiah berarti untuk memutus belakang. Kata dabiru dari segi bahasa berarti belakang. Biasanya, yang ada di belakang / dabiru tidak terkejar atau dibiarkan pergi karena pengejarnya sudah lelah dan bosan sehingga yang di belakang itu dapa

In [4]:
list_query

['agama',
 'al-abrar',
 'al-aqsha',
 'al-barr',
 'al-ghafur',
 'al-haq',
 'al-izzah',
 'al-jabbar',
 'al-lathif',
 'al-muhaimin',
 'al-mutakabbir',
 'al-qahir',
 'al-quddus',
 'al-yamin',
 'an-nas',
 'api',
 'ar-rahman',
 'ashr',
 'at-taghabun',
 'aula',
 'badi',
 'bashir',
 'berkah',
 'bintang',
 'buah',
 'dabiru',
 'fasik',
 'harapan',
 'hasad',
 'ismuhu ahmad',
 'kalimat',
 'khabir',
 'khusyu',
 'malaikat',
 'mati',
 'mimpi',
 'nafsu',
 'nasuhan',
 'pencuri',
 'quraisy',
 'ramadhan',
 'riba',
 'saudara',
 'shirath',
 'syafaat',
 'syariat',
 'tahajud',
 'tanda',
 'wakil',
 'yajuj dan majuj']

## Load Result File

In [5]:
result_file_name = 'result/result_file_22-5-2020-17-31.csv'
result_df = pd.read_csv(result_file_name)
result_df = result_df.sort_values(by = "Query", axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
result_df.head()

,Query,Total,Similarity Queries,Rank Documents,Summarization
0,Agama\n,15,"{'2-132-2.txt': 0.5180908684653747, '2-130-1.t...","[(0.10243279963337978, '2-132-2.txt', ['Memang...","Memang banyak agama yang dikenal oleh manusia,..."
26,Al-abrar\n,10,"{'76-4-9.txt': 0.6279644730092272, '76-4-4.txt...","[(0.16384370541524665, '76-4-4.txt', ['Makna',...",Makna kedua dari kata al-abrar adalah daratan....
45,Al-aqsha\n,10,"{'17-1-1.txt': 0.5, '17-1-3.txt': 0.3333333333...","[(0.15283784551725724, '17-1-9.txt', ['Penamaa...",Penamaan Masjid terjauh merupakan isyarat bahw...
35,Al-barr\n,10,"{'52-28-3.txt': 0.5, '52-28-10.txt': 0.4736067...","[(0.15204891127982148, '52-28-2.txt', ['Terdap...",Terdapat sekian makna yang dikandung oleh kata...
42,Al-ghafur\n,10,"{'18-59-10.txt': 0.5834890079292623, '18-59-1....","[(0.14482511817624363, '18-59-10.txt', ['Imam'...","Imam Ghazali, dalam membedakan sifat Al-Ghafur..."


In [6]:
queries = result_df['Query'].str.replace('\n','').str.lower().str.strip() # get clean queries
queries.head()


0         agama
26     al-abrar
45     al-aqsha
35      al-barr
42    al-ghafur
Name: Query, dtype: object

In [7]:
'agama'=='agama'

True

## Compare Sentences

In [8]:
def find_index_query(queries_df, query):
    """
    find index of queries data frame with query
    return: index
    """
    series = queries_df.str.find(query)
    for index, value in series.items():
        if value >= 0:
            return index

    return -1

In [9]:
# calculate f-score
def get_fscore(result_sentences, target_sentences):
    """
    Precision(P) is the number of sentences occurring in both system and ideal summaries divided by the number of sentences in the system summary
    Recall(R) is the numberof sentences occurring in both system and ideal summaries divided by the number ofsentences in the ideal summary
    """
    # get number of sentences
    num_res_sentences = len(result_sentences)
    num_target_sentences = len(target_sentences)

    # find same sentence 
    num_occur_sentences = 0
    for res_sentence in result_sentences:
        if res_sentence in target_sentences:
            num_occur_sentences += 1

    try:
        precision = num_occur_sentences / num_res_sentences
        recall = num_occur_sentences / num_target_sentences
        f_score = 2 * precision * recall / (precision + recall)
        
        return f_score
    
    except ZeroDivisionError:
        return 0

   

In [10]:
# iterate list query
f_scores = []
for query in list_query:
    
    # find index of query
    index_df = find_index_query(queries,query)
    
    # get result and target summarization
    result_summarization = result_df.loc[index_df]['Summarization']
    target_summarization = list_documents[query]

    # split it to token
    result_sentences = sent_tokenize(result_summarization)
    target_sentences = sent_tokenize(target_summarization)

    # get precision
    f_score = get_fscore(result_sentences, target_sentences)
    f_scores.append(f_score)
    # Debug
    print("\nQuery : {} \nTarget : \n{} \nResult : \n{} \nF-Score : {}".format(query,target_summarization,result_summarization,f_score))




Query : agama 
Target : 
Kata millah biasanya diartikan agama. Di sisi lain, kata millah biasanya digunakan untuk menunjuk kepada sekumpulan ajaran, berbeda dengan kata dîn/ agama yang dapat digunakan untuk menunjuk kepada satu atau beberapa perinciannya. Maksudnya agama ini adalah tuntunan Allah, bukan ciptaanku. Memang banyak agama yang dikenal oleh manusia, tetapi yang ini, yakni yang intinya adalah penyerahan diri secara mutlak kepada Allah, itulah yang direstui dan dipilih-Nya. Karena itu, maka janganlah kamu mati kecuali kamu dalam keadaan berserah diri kepada Allah, yakni memeluk agama islam. Din / agama mempunyai banyak arti, antara lain ketundukan, ketaatan, perhitungan, balasan. Juga berarti agama karena dengan agama seseorang bersikap tunduk dan taat serta akan diperhitungkan seluruh amalnya, yang atas dasar itü ia memeroleh balasan dan ganjaran. Sesungguhnya agama yang disyariatkan di sisi Allah adalah İslam, demikian terjemahan yang populer. Agama, atau ketaatan kepada-Ny

In [11]:
# Average
average_f_score = np.average(f_scores)
print("\nAverage F-Score: {} ".format(average_f_score))


Average F-Score: 0.617350429801298 
